In [ ]:
import pandas as pd

from FranconianWeather import download_file
from FranconianWeather import load_txt_from_zip
from FranconianWeather import save_to_sqlite

In [ ]:
bamberg = "monatswerte_KL_00282_18810101_20231231_hist.zip"
wuerzburg = "monatswerte_KL_05705_18810101_20231231_hist.zip"

download_file(bamberg)
download_file(wuerzburg)

In [ ]:
# Define the name of the text file inside the zip file
bamberg_data_file = "produkt_klima_monat_18810101_20231231_00282.txt"

# Load the text file into a pandas DataFrame
bamberg_df = load_txt_from_zip(bamberg, bamberg_data_file)

bamberg_df

In [ ]:
wuerzburg_data_file = "produkt_klima_monat_18810101_20231231_05705.txt"

wuerzburg_df = load_txt_from_zip(wuerzburg, wuerzburg_data_file)

wuerzburg_df

In [ ]:
# Combine the dataframes so we can save to a database
combined_df = pd.concat([bamberg_df, wuerzburg_df])

In [ ]:
# Save the DataFrame to an SQLite database
save_to_sqlite(combined_df, 'franconian-weather')